In [ ]:
# OpenPose BODY_25 keypoints.
# //     {0,  "Nose"},
# //     {1,  "Neck"},
# //     {2,  "RShoulder"},
# //     {3,  "RElbow"},
# //     {4,  "RWrist"},
# //     {5,  "LShoulder"},
# //     {6,  "LElbow"},
# //     {7,  "LWrist"},
# //     {8,  "MidHip"},
# //     {9,  "RHip"},
# //     {10, "RKnee"},
# //     {11, "RAnkle"},
# //     {12, "LHip"},
# //     {13, "LKnee"},
# //     {14, "LAnkle"},
# //     {15, "REye"},
# //     {16, "LEye"},
# //     {17, "REar"},
# //     {18, "LEar"},
# //     {19, "LBigToe"},
# //     {20, "LSmallToe"},
# //     {21, "LHeel"},
# //     {22, "RBigToe"},
# //     {23, "RSmallToe"},
# //     {24, "RHeel"},
# //     {25, "Background"}

In [ ]:
import os
import json
import re
import numpy as np
from math import degrees, atan, acos, asin

### EDIT BELOW ###
IMAGES_DIR = "C:/Users/adada/Documents/Python workspace/R&I/input"
OUTPUT_DIR = "C:/Users/adada/Documents/Python workspace/R&I/output"
OPENPOSE_DIR = "C:/Users/adada/Documents/Python workspace/R&I/openpose-old"

IDEAL_NECK_NOSE_ANGLE = 150
IDEAL_NECK_SHOULDER_ANGLE = 180
MARGIN_PERCENT = 10

### DON'T EDIT BELOW ###
# Calculate distance between two BODY_25 keypoints.
def calculateDistance(k1, k2):
    distance = np.sqrt(pow(k2['x']-k1['x'], 2)+pow(k2['y']-k1['y'], 2))
    return distance

# Calculate angles in a triangle. NOTE: distances are specified in the correct order.
def calculateAngles(d1, d2, d3):
    d1d2 = degrees(acos((pow(d1, 2)+pow(d2, 2)-pow(d3, 2))/(2*d1*d2)))
    d2d3 = degrees(acos((pow(d2, 2)+pow(d3, 2)-pow(d1, 2))/(2*d2*d3)))
    d3d1 = degrees(acos((pow(d3, 2)+pow(d1, 2)-pow(d2, 2))/(2*d3*d1)))
    return {
        'd1d2': d1d2,
        'd2d3': d2d3,
        'd3d1': d3d1
    }

# Extract and properly format BODY_25 keypoints.
def formatPersonPose(poseKeypoints):
    person = {}
    if (len(poseKeypoints)/3 != 25):
        print("Invalid keypoints")
        return;
    for i in range(0, 25):
        index = i*3
        keypoint = {
            'x': poseKeypoints[index],
            'y': poseKeypoints[index+1],
            'c': poseKeypoints[index+2]
        }
        person[i] = keypoint
    return person

# Compute OpenPose analysis.
def computePose():
    # Compute OpenPose detection.
    stream = os.system('cd "' + OPENPOSE_DIR + '" && "./bin/OpenPoseDemo.exe" --image_dir "' + IMAGES_DIR + '" --write_images "' + OUTPUT_DIR + '" --write_json "' + OUTPUT_DIR + '" --display 0')
    if (stream != 0):
        print('OpenPose detection failed.')
        return None

    # Get output json files.
    regex = re.compile('(.*json$)')
    extractedFiles = []
    for root, dirs, files in os.walk(OUTPUT_DIR):
        for file in files:
            if regex.match(file):
                extractedFiles.append(file)
    return extractedFiles

# Extract json object to specified file.
def extractJsonFromFile(file):
    extractedJson = None
    with open(OUTPUT_DIR + "/" + file) as f:
        extractedJson = json.load(f)
    return extractedJson

# Extract persons keypoints from specified json object.
def extractPersonsKeypointsFromJson(jsonObject):
    persons = []
    for person in jsonObject['people']:
        persons.append(formatPersonPose(person['pose_keypoints_2d']))
    return persons
            
# Analyze human pose with specified person keypoints.
def analysePose(person):
    # Extract required distances
    noseToNeck = calculateDistance(person[0], person[1])
    neckToMidHip = calculateDistance(person[1], person[8])
    midHipToNose = calculateDistance(person[8], person[0])

    neckToRshoulder = calculateDistance(person[1], person[2])
    rshoulderToLshoulder = calculateDistance(person[2], person[5])
    lshoulderToNeck = calculateDistance(person[5], person[1])

    # Calculate angles
    neckNoseAngle = calculateAngles(noseToNeck, neckToMidHip, midHipToNose)['d1d2']
    shoulderNeckAngle = calculateAngles(neckToRshoulder, rshoulderToLshoulder, lshoulderToNeck)['d3d1']

    isNeckNoseAngleCorrect = (neckNoseAngle >= IDEAL_NECK_NOSE_ANGLE * float(1 - MARGIN_PERCENT/100)) and (neckNoseAngle <= IDEAL_NECK_NOSE_ANGLE * float(1 + MARGIN_PERCENT/100))
    isShoulderNeckAngleCorrect = (shoulderNeckAngle >= IDEAL_NECK_SHOULDER_ANGLE * float(1 - MARGIN_PERCENT/100)) and (shoulderNeckAngle <= IDEAL_NECK_SHOULDER_ANGLE * float(1 + MARGIN_PERCENT/100))
    
    print("Angle between nose and neck: " + str(neckNoseAngle) + "; Valid: " + str(isNeckNoseAngleCorrect))
    print("Angle between shoulder and neck: " + str(shoulderNeckAngle) + "; Valid: " + str(isShoulderNeckAngleCorrect))
    if (isNeckNoseAngleCorrect and isShoulderNeckAngleCorrect):
        print("Valid pose.")
    else:
        print("Invalid pose.")

In [ ]:
files = computePose()

for file in files:
    print("\nAnalysing file named: " + file)
    # Extract json
    jsonObject = extractJsonFromFile(file)
    # Extract persons
    persons = extractPersonsKeypointsFromJson(jsonObject)
    # Analyse persons
    counter = 1
    for person in persons:
        print("Analysing detected person n°" + str(counter))
        # Analyse person
        analysePose(person) 
        counter = counter + 1